In [1]:
from docx import Document
import requests
import csv
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

document = Document()

API_KEY = os.environ.get("API_KEY")

OFFICER_NAME = os.environ.get("OFFICER_NAME")
OFFICER_DOB = os.environ.get("OFFICER_DOB")

URL = f"https://api.company-information.service.gov.uk/search/officers?q=\"{OFFICER_NAME}\""

headers = {"Authorization": API_KEY}

print(API_KEY, OFFICER_NAME, OFFICER_DOB)

eb613ae5-33b1-4aaf-8f23-18ad86769709 Keith Ranger DOLLIVER 1963-01


In [3]:
# Add a title to the document
document.add_heading(f"Associated companies for: {OFFICER_NAME}", level=1)
current_heading = document.add_heading('Current appointments', level=1)
current_end = document.add_paragraph('-')
former_heading = document.add_heading('Former appointments', level=1)
former_end = document.add_paragraph('--')

document

In [4]:
# Load the SIC codes and descriptions from the CSV file
with open("SIC07_CH_condensed_list_en.csv", newline='') as csvfile:
    sic_codes_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    sic_mapping = {row[0]: row[1] for row in sic_codes_reader}

    sic_mapping

In [7]:
# Make the API request
response = requests.get(URL, headers=headers)
data = response.json()

data

{'items_per_page': 20,
 'kind': 'search#officers',
 'start_index': 0,
 'items': [{'date_of_birth': {'month': 1, 'year': 1963},
   'title': 'Keith Ranger DOLLIVER',
   'description_identifiers': ['appointment-count', 'born-on'],
   'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
   'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
   'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
   'address': {'region': 'Usa',
    'locality': 'Washington 98006',
    'address_line_2': 'Bellevue',
    'address_line_1': '5580 179th Avenue Se'},
   'description': 'Total number of appointments 1 - Born January 1963',
   'kind': 'searchresults#officer',
   'snippet': '',
   'appointment_count': 1},
  {'appointment_count': 1,
   'description': 'Total number of appointments 1 - Born January 1963',
   'kind': 'searchresults#officer',
   'snippet': '',
   'address': {'premises': 'One Microsoft Way',
    'address_line_1': 'Redmond',
    'country': 'U

In [8]:
# only keep officers with exact name match
exact_name_matches = []
for item in data['items']:
    if item['title'].lower() == OFFICER_NAME.lower():
        exact_name_matches.append(item)

exact_name_matches

[{'date_of_birth': {'month': 1, 'year': 1963},
  'title': 'Keith Ranger DOLLIVER',
  'description_identifiers': ['appointment-count', 'born-on'],
  'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
  'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
  'address': {'region': 'Usa',
   'locality': 'Washington 98006',
   'address_line_2': 'Bellevue',
   'address_line_1': '5580 179th Avenue Se'},
  'description': 'Total number of appointments 1 - Born January 1963',
  'kind': 'searchresults#officer',
  'snippet': '',
  'appointment_count': 1},
 {'appointment_count': 1,
  'description': 'Total number of appointments 1 - Born January 1963',
  'kind': 'searchresults#officer',
  'snippet': '',
  'address': {'premises': 'One Microsoft Way',
   'address_line_1': 'Redmond',
   'country': 'United States',
   'locality': 'Wa 98052'},
  'links': {'self': '/officers/mI2cWvy4QIMWxs3EvB7Es6FrXG

In [12]:
exact_name_dob_matches = []
for item in data['items']:
    if (
        item['title'].lower() == OFFICER_NAME.lower() #if name matches
        and isinstance(item.get('date_of_birth'), dict) #and there's a date of birth
        and str(item['date_of_birth']['year']) #and that dob formatted is equal to the officer's
        +'-'
        +str(item['date_of_birth']['month']).zfill(2) 
        == OFFICER_DOB[:7]
    ):
        exact_name_dob_matches.append(item) #add to the list

exact_name_dob_matches

[{'date_of_birth': {'month': 1, 'year': 1963},
  'title': 'Keith Ranger DOLLIVER',
  'description_identifiers': ['appointment-count', 'born-on'],
  'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
  'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
  'address': {'region': 'Usa',
   'locality': 'Washington 98006',
   'address_line_2': 'Bellevue',
   'address_line_1': '5580 179th Avenue Se'},
  'description': 'Total number of appointments 1 - Born January 1963',
  'kind': 'searchresults#officer',
  'snippet': '',
  'appointment_count': 1},
 {'appointment_count': 1,
  'description': 'Total number of appointments 1 - Born January 1963',
  'kind': 'searchresults#officer',
  'snippet': '',
  'address': {'premises': 'One Microsoft Way',
   'address_line_1': 'Redmond',
   'country': 'United States',
   'locality': 'Wa 98052'},
  'links': {'self': '/officers/mI2cWvy4QIMWxs3EvB7Es6FrXG

In [10]:
data['items']

[{'date_of_birth': {'month': 1, 'year': 1963},
  'title': 'Keith Ranger DOLLIVER',
  'description_identifiers': ['appointment-count', 'born-on'],
  'links': {'self': '/officers/hgf_hGSfHEbv9YaGnc7D8ZW6Ojo/appointments'},
  'matches': {'snippet': [], 'title': [1, 5, 7, 12, 14, 21]},
  'address_snippet': '5580 179th Avenue Se, Bellevue, Washington 98006, Usa',
  'address': {'region': 'Usa',
   'locality': 'Washington 98006',
   'address_line_2': 'Bellevue',
   'address_line_1': '5580 179th Avenue Se'},
  'description': 'Total number of appointments 1 - Born January 1963',
  'kind': 'searchresults#officer',
  'snippet': '',
  'appointment_count': 1},
 {'appointment_count': 1,
  'description': 'Total number of appointments 1 - Born January 1963',
  'kind': 'searchresults#officer',
  'snippet': '',
  'address': {'premises': 'One Microsoft Way',
   'address_line_1': 'Redmond',
   'country': 'United States',
   'locality': 'Wa 98052'},
  'links': {'self': '/officers/mI2cWvy4QIMWxs3EvB7Es6FrXG

In [13]:
# Loop through the exact name matches and make a request to each URL in the links dictionary
for match in exact_name_dob_matches:
    officer_url = f"https://api.company-information.service.gov.uk{match['links']['self']}"
    officer_response = requests.get(officer_url, headers=headers)
    officer_data = officer_response.json()



    # Loop through the officer's appointments and print the company name, number, and nature of business
    for appointment in officer_data['items']:
        company_name = appointment['appointed_to']['company_name']
        company_number = appointment['appointed_to']['company_number']
        appointed_on = appointment['appointed_on']
        officer_role = appointment['officer_role']
        company_status = appointment['appointed_to']['company_status']
        psc_name = ''
        company_title = f"{company_name} ({company_number})"

        # Get the company profile URL
        company_profile_url = f"https://api.company-information.service.gov.uk/company/{company_number}"
        company_profile_response = requests.get(company_profile_url, headers=headers)
        company_profile_data = company_profile_response.json()

        # Get the SIC code and look up the activity in the mapping dictionary
        sic_code = company_profile_data.get('sic_codes', ["N/A"])[0]
        activity = sic_mapping.get(sic_code, "Unknown")

        #Get company status, incorporation date, dissolution date

        company_inc = company_profile_data.get('date_of_creation')
        if 'date_of_cessation' in company_profile_data:
            company_dis = company_profile_data['date_of_cessation']
        else:
            company_dis = None

        # Get the company's persons with significant control data
        company_psc_url = f"https://api.company-information.service.gov.uk/company/{company_number}/persons-with-significant-control"
        company_psc_response = requests.get(company_psc_url, headers=headers)
        company_psc_data = company_psc_response.json()

        #print(json.dumps(company_psc_data, indent=4))

        # Loop through the company's persons with significant control and print the name
        psc_names = []
        if 'items' in company_psc_data and company_psc_data['items']:
            for psc in company_psc_data['items']:
                if 'name' in psc:
                    psc_name = psc['name']
                    psc_names.append(psc_name)
        else:
            print("No PSC data available for this company.")

        if "resigned_on" in appointment:
            resigned_on = appointment["resigned_on"]
            if len(psc_names) == 1:
                for psc_name in psc_names:
                    psc_statement = f"The company has a person with significant control named {psc_name}."
                new_paragraph = former_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} was appointed {officer_role} of {company_name} ({company_number}) on {appointed_on} and resigned on {resigned_on}. The nature of business is {activity}. {psc_statement}")


            elif len(psc_names) > 1:
                last_name = psc_names.pop()
                full_list = ", ".join(psc_names)
                psc_statement = f"The company has the following persons with significant control: {full_list} and {last_name}."
                new_paragraph = former_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} was appointed {officer_role} of {company_name} ({company_number}) on {appointed_on} and resigned on {resigned_on}. The nature of business is {activity}. {psc_statement}")


            else:
                psc_statement = (
                    "The company has no persons with significant control"
                )
                new_paragraph = former_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} was appointed {officer_role} of {company_name} on {appointed_on} and resigned on {resigned_on}. The nature of business is {activity}. {psc_statement}")

        else:
            if len(psc_names) == 1:
                for psc_name in psc_names:
                    psc_statement = f"The company has a person with significant control named {psc_name}."
                    if 'active' in company_status:
                        new_paragraph = current_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} has been serving as {officer_role} of {company_name} since {appointed_on}. The nature of business is {activity}. {psc_statement}"
                )
                    else:
                        new_paragraph = former_end.insert_paragraph_before(
                            f"{company_name} ({company_number}) \n{OFFICER_NAME} served as {officer_role} of {company_name} between {appointed_on} and {company_dis}. The nature of business was {activity}. {psc_statement}"
                        )
            elif len(psc_names) > 1:
                last_name = psc_names.pop()
                full_list = ", ".join(psc_names)
                psc_statement = f"The company has the following persons with significant control: {full_list} and {last_name}."
                if 'active' in company_status:
                    new_paragraph = current_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} has been serving as {officer_role} of {company_name} since {appointed_on}. The nature of business is {activity}. {psc_statement}"
                )
                else:
                    new_paragraph = former_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} served as {officer_role} of {company_name} between {appointed_on} and {company_dis}. The nature of business is {activity}. {psc_statement}"
                    )

            else:
                psc_statement = (
                    "The company has no persons with significant control"
                )
                if 'active' in company_status:
                    new_paragraph = current_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} has been serving as {officer_role} of {company_name} since {appointed_on}. The nature of business is {activity}. {psc_statement}"
                )
                else:
                    new_paragraph = former_end.insert_paragraph_before(
                    f"{company_name} ({company_number}) \n{OFFICER_NAME} served as {officer_role} of {company_name} between {appointed_on} and {company_dis}. The nature of business was {activity}. {psc_statement}"
                    )

No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.
No PSC data available for this company.


In [ ]:
# Save the document as a Word file
document.save(f'Associated companies for {OFFICER_NAME}.docx')